In [14]:
#pip install torch torchvision torchaudio
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [15]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0.,1.])
dataset = TensorDataset(inputs,labels)
dataloader = DataLoader(dataset)

In [25]:
class LightningLSTM(L.LightningModule):

    def __init__(self):

        super().__init__()

        self.lstm = nn.LSTM(input_size = 1, hidden_size = 1)


    def forward(self, input):

        input_trans = input.view(len(input),1)

        lstm_out, temp = self.lstm(input_trans)

        prediction = lstm_out[-1]

        return(prediction)
    
    def configure_optimizers(self):
        
        return(Adam(self.parameters(), lr=0.1))

    def training_step(self, batch, batch_idx):

        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i)**2

        self.log("train_loss", loss)

        if(label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)

        return(loss)

In [26]:
model = LightningLSTM()
print("\nNow let's compare the observed and predicted values....")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values....
Company A: Observed = 0, Predicted = tensor([-0.4205])
Company B: Observed = 1, Predicted = tensor([-0.4204])


In [27]:
trainer = L.Trainer(max_epochs = 300, log_every_n_steps=2)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 121.93it/s, v_num=10]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 92.12it/s, v_num=10] 


In [28]:
print("\nNow let's compare the observed and predicted values....")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values....
Company A: Observed = 0, Predicted = tensor([-2.3683e-06])
Company B: Observed = 1, Predicted = tensor([0.9825])
